SCRAPPING

In [4]:
# =================================================================
# LANGKAH 1: Instal library yt-dlp dan import pandas
# =================================================================
!pip install yt-dlp
import subprocess
import json
import datetime
import pandas as pd  # Library untuk DataFrames dan menyimpan ke CSV

# =================================================================
# LANGKAH 2: Tentukan URL Video
# =================================================================
YOUTUBE_URL = "https://youtu.be/YbTBIIoAz74?si=k4VtgJMaP9ROuPJP"
OUTPUT_FILENAME = "youtube_metadata.csv"
COMMENTS_OUTPUT_FILENAME = "youtube_comments.csv" # Nama file untuk komentar

print(f"Memproses URL: {YOUTUBE_URL}\n")

# =================================================================
# LANGKAH 3: Ekstrak Metadata dan Komentar
# =================================================================
try:
    # Menjalankan perintah yt-dlp dengan opsi '-j' untuk mendapatkan output JSON
    # Menambahkan opsi '--skip-download --get-comments' untuk mendapatkan komentar tanpa mengunduh video
    result = subprocess.run(
        ['yt-dlp', '-j', '--skip-download', '--get-comments', YOUTUBE_URL],
        capture_output=True,
        text=True,
        check=True
    )

    # Memuat output JSON
    video_data = json.loads(result.stdout)

    # Memformat data metadata
    upload_date_raw = video_data.get('upload_date')
    upload_date = datetime.datetime.strptime(upload_date_raw, '%Y%m%d').strftime('%d %B %Y')

    # Data metadata yang akan dimasukkan ke CSV
    data_for_csv = {
        'Video_ID': [video_data.get('id')],
        'Judul': [video_data.get('title')],
        'Uploader': [video_data.get('uploader')],
        'Tanggal_Upload': [upload_date],
        'Durasi_Detik': [video_data.get('duration')],
        'Jumlah_Penonton': [video_data.get('view_count')],
        'Jumlah_Suka': [video_data.get('like_count')],
        'Deskripsi': [video_data.get('description')]
    }

    # =================================================================
    # LANGKAH 4: Konversi Metadata ke DataFrame dan Simpan ke CSV
    # =================================================================
    df_metadata = pd.DataFrame(data_for_csv)

    # Menyimpan DataFrame metadata ke berkas CSV
    df_metadata.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')

    print("=====================================================")
    print(f"✅ Data metadata berhasil diekstraksi dan disimpan ke: **{OUTPUT_FILENAME}**")
    print("=====================================================")
    print("\nBerikut pratinjau data metadata yang disimpan:")
    print(df_metadata.head())


    # =================================================================
    # LANGKAH 5: Ekstrak dan Simpan Komentar
    # =================================================================
    comments_data = video_data.get('comments', []) # Mengambil daftar komentar, default ke list kosong jika tidak ada

    if comments_data:
        # Mengambil informasi yang relevan dari setiap komentar
        comment_list = []
        for comment in comments_data:
            comment_list.append({
                'Comment_ID': comment.get('id'),
                'Author': comment.get('author'),
                'Text': comment.get('text'),
                'Timestamp': comment.get('timestamp'),
                'Likes': comment.get('like_count'),
                'Is_Replied_To_Author': comment.get('is_replied_to_author')
            })

        df_comments = pd.DataFrame(comment_list)

        # Menyimpan DataFrame komentar ke berkas CSV
        df_comments.to_csv(COMMENTS_OUTPUT_FILENAME, index=False, encoding='utf-8')

        print("\n=====================================================")
        print(f"✅ Data komentar berhasil diekstraksi dan disimpan ke: **{COMMENTS_OUTPUT_FILENAME}**")
        print("=====================================================")
        print("\nBerikut pratinjau data komentar yang disimpan:")
        display(df_comments.head()) # Menggunakan display untuk tabel yang lebih baik
        print("\nUntuk mengunduh file, cari 'youtube_comments.csv' di panel File Colab.")
    else:
        print("\n⚠️ Tidak ada komentar yang ditemukan untuk video ini atau scraping komentar dinonaktifkan.")


except subprocess.CalledProcessError as e:
    print(f"❌ ERROR: Gagal menjalankan yt-dlp.")
    print(f"Pesan Error: {e.stderr}")
except Exception as e:
    print(f"❌ Terjadi kesalahan: {e}")

Memproses URL: https://youtu.be/YbTBIIoAz74?si=k4VtgJMaP9ROuPJP

✅ Data metadata berhasil diekstraksi dan disimpan ke: **youtube_metadata.csv**

Berikut pratinjau data metadata yang disimpan:
      Video_ID                                              Judul  \
0  YbTBIIoAz74  FERRY IRWANDI, OM SAMPAIKAN KE GIBRAN INI‼️OM ...   

          Uploader Tanggal_Upload  Durasi_Detik  Jumlah_Penonton  Jumlah_Suka  \
0  Deddy Corbuzier  30 April 2025          4525          2829884        40872   

                                           Deskripsi  
0  #ferryirwandi #gibran #podcast \nJoin to see u...  

✅ Data komentar berhasil diekstraksi dan disimpan ke: **youtube_comments.csv**

Berikut pratinjau data komentar yang disimpan:


,Comment_ID,Author,Text,Timestamp,Likes,Is_Replied_To_Author
0,Ugx_mbo0SBB4VrAR74l4AaABAg,@GIBRANN-l5w,jj,1760659200,0,None
1,Ugzj0CdilHTuf2Zb_-Z4AaABAg,@febriant6902,ngomong sm ferry....sama aje melucuti pertahan...,1760659200,0,None
2,Ugys8tO4Cy6r6qQYbHN4AaABAg,@desyidawaty4961,"Dua orang brilliant.\nAku suka mendengarkan, k...",1759449600,0,None
3,UgyEC6-7dofHCh-Sw7B4AaABAg,@indratamtama5871,hei ded.... \nklo MBG beracun kayak skrmg gini...,1759190400,0,None
4,Ugzt4WfGEt5ZwemOodt4AaABAg,@tarcisiusslamet8050,Ini rupanya feri satu grombolan dg Roisoryo,1759190400,0,None



Untuk mengunduh file, cari 'youtube_comments.csv' di panel File Colab.


PREPROCESSING

In [8]:
# [Pastikan kode instalasi dan definisi fungsi preprocessing di atas sudah dijalankan]
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Setup NLTK resources (jika belum)
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords', quiet=True)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt', quiet=True)

indo_stopwords = set(stopwords.words('indonesian'))

# --- ASUMSI DATA df_comments SUDAH ADA DARI SEL SEBELUMNYA ---
# GANTIKAN INI DENGAN DATA ASLI ANDA JIKA df_comments BELUM TERDEFINISI
# SIMULASI DATA JIKA df_comments BELUM TERDEFINISI
try:
    df = df_comments.copy()
except NameError:
    print("⚠️ Data 'df_comments' tidak ditemukan. Menggunakan data simulasi.")
    data = {
        'Comment_ID': [1, 2, 3, 4, 5],
        'Author': ['Budi', 'Citra', 'Dian', 'Eko', 'Fani'],
        'Text': [
            'Waduh, videonya keren bgt 👍. Tks bnyk untuk infonya!! (cek link di bio) https://link.com',
            'Aku kurang setuju sih dgn poin 3. Ada pendapat lain??? @budi',
            'Kontennya bagus, tapi bahasanya agak susah dimengerti. #saran',
            'Kapan upload lagi min? Saya tunggu!!11!!!',
            'Ini mah gampang bgt. Gaada tantangannya. @citra'
        ],
        'Timestamp': ['2025-10-30 10:00:00', '2025-10-30 11:00:00', '2025-10-30 12:00:00', '2025-10-30 13:00:00', '2025-10-30 14:00:00'],
        'Likes': [15, 3, 7, 21, 1],
        'Is_Replied_To_Author': [False, True, False, False, True]
    }
    df = pd.DataFrame(data)

# --- (Definisi semua fungsi preprocessing Anda harus ditempatkan di sini) ---
# --- (Asumsi semua fungsi remove_symbols, case_folding, dll. sudah didefinisikan) ---

def remove_symbols(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    return text

def case_folding(text):
    if not isinstance(text, str): return ""
    return text.lower()

def remove_punc_numbers(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def normalize_text(text):
    if not isinstance(text, str): return ""
    slang_dict = {'bgt': 'banget', 'tks': 'terima kasih', 'dgn': 'dengan', 'min': 'admin', 'gaada': 'tidak ada', 'udh': 'sudah'}
    words = text.split()
    normalized_words = [slang_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

def remove_stopwords(text):
    if not isinstance(text, str): return ""
    words = text.split()
    filtered_words = [word for word in words if word not in indo_stopwords]
    return ' '.join(filtered_words)

def tokenize(text):
    if not isinstance(text, str): return []
    return word_tokenize(text)

# --- APLIKASI SELURUH LANGKAH PREPROCESSING (SESUAI DENGAN KODE ANDA) ---
df['Text_Cleaned'] = df['Text'].copy()
df['Text_Cleaned'] = df['Text_Cleaned'].apply(remove_symbols)
df['Text_Cleaned'] = df['Text_Cleaned'].apply(case_folding)
df['Text_Cleaned'] = df['Text_Cleaned'].apply(remove_punc_numbers)
df['Text_Cleaned'] = df['Text_Cleaned'].apply(normalize_text)
df['Text_Cleaned'] = df['Text_Cleaned'].apply(remove_stopwords)
df['Text_Tokenized'] = df['Text_Cleaned'].apply(tokenize)

# --- LANGKAH BARU: SIMPAN KE CSV ---
OUTPUT_FILENAME = "komentar_bersih.csv"

# Konversi kolom list (Text_Tokenized) menjadi string agar bisa disimpan rapi di CSV
df['Text_Tokenized_Str'] = df['Text_Tokenized'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Pilih kolom yang akan disimpan (semua kolom asli + kolom hasil preprocessing)
columns_to_save = list(df.columns)
# Kita akan menghapus kolom list 'Text_Tokenized' dan menggunakan versi stringnya
if 'Text_Tokenized' in columns_to_save:
    columns_to_save.remove('Text_Tokenized')

# Simpan DataFrame ke berkas CSV
df[columns_to_save].to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')

# --- HASIL AKHIR ---
print("\n" + "=" * 50)
print(f"✅ Pra-pemrosesan selesai! Data berhasil disimpan ke: **{OUTPUT_FILENAME}**")
print("Kolom 'Text_Tokenized' disimpan sebagai string di 'Text_Tokenized_Str' untuk kompatibilitas CSV.")
print("=" * 50)
print("\nPratinjau data yang disimpan:")
print(df[['Text', 'Text_Cleaned', 'Text_Tokenized_Str']].head())


✅ Pra-pemrosesan selesai! Data berhasil disimpan ke: **komentar_bersih.csv**
Kolom 'Text_Tokenized' disimpan sebagai string di 'Text_Tokenized_Str' untuk kompatibilitas CSV.

Pratinjau data yang disimpan:
                                                Text  \
0                                                 jj   
1  ngomong sm ferry....sama aje melucuti pertahan...   
2  Dua orang brilliant.\nAku suka mendengarkan, k...   
3  hei ded.... \nklo MBG beracun kayak skrmg gini...   
4       Ini rupanya feri satu grombolan dg  Roisoryo   

                                        Text_Cleaned  \
0                                                 jj   
1  ngomong sm ferrysama aje melucuti pertahanan k...   
2          orang brilliant suka mendengarkan bermutu   
3  hei ded klo mbg beracun kayak skrmg gini loe s...   
4                         feri grombolan dg roisoryo   

                                  Text_Tokenized_Str  
0                                                 jj  
1  ngomong

TF IDF

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import re

# Nama file CSV yang akan dimuat
FILE_CSV = 'data_TF_IDF.csv'

# =================================================================
# LANGKAH 1: MEMUAT DATA DAN PREPARASI KORPUS
# =================================================================

try:
    # Memuat data dari file CSV
    df = pd.read_csv(FILE_CSV)
    print(f"✅ File '{FILE_CSV}' berhasil dimuat. Jumlah komentar: {len(df)}")
except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_CSV}' tidak ditemukan.")
    print("Harap unggah file tersebut ke sesi Colab Anda sebelum melanjutkan.")
    # Membuat data simulasi jika file tidak ada
    df = pd.DataFrame({
        'Text_Cleaned': [
            "video keren banget terima kasih info",
            "kurang setuju poin pendapat lain",
            "konten bagus bahasa dimengerti saran",
            "admin tunggu upload",
            "gampang banget tantangan"
        ]
    })
    print("Menggunakan data simulasi untuk demonstrasi.")

# Pastikan hanya menggunakan baris dengan data di 'Text_Cleaned'
df = df.dropna(subset=['Text_Cleaned'])
df['Text_Cleaned'] = df['Text_Cleaned'].astype(str)

# Korpus adalah daftar semua komentar yang sudah bersih
korpus = df['Text_Cleaned'].tolist()

print("\n--- Pratinjau 5 Komentar Bersih yang Digunakan ---")
for i, text in enumerate(korpus[:5]):
    print(f"Komentar {i+1}: {text}")
print("-" * 50)

# Unduh stopwords Bahasa Indonesia jika belum
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords', quiet=True)

# Dapatkan daftar stopwords Bahasa Indonesia
list_stopwords = stopwords.words('indonesian')

# Tambahkan kata-kata umum yang mungkin lolos (jika diperlukan)
# Misalnya: 'banget' jika Anda tidak ingin menganggapnya penting
# list_stopwords.extend(['banget', 'sih', 'ya'])

❌ ERROR: File 'data_TF_IDF.csv' tidak ditemukan.
Harap unggah file tersebut ke sesi Colab Anda sebelum melanjutkan.
Menggunakan data simulasi untuk demonstrasi.

--- Pratinjau 5 Komentar Bersih yang Digunakan ---
Komentar 1: video keren banget terima kasih info
Komentar 2: kurang setuju poin pendapat lain
Komentar 3: konten bagus bahasa dimengerti saran
Komentar 4: admin tunggu upload
Komentar 5: gampang banget tantangan
--------------------------------------------------


In [10]:
# =================================================================
# LANGKAH 2: TERM FREQUENCY (TF) MENGGUNAKAN CountVectorizer
# =================================================================

# Inisialisasi CountVectorizer dengan stopwords Bahasa Indonesia
# CountVectorizer secara otomatis melakukan tokenisasi dasar dan case folding
cv = CountVectorizer(stop_words=list_stopwords)

# Terapkan CountVectorizer ke korpus (teks)
word_count_matrix = cv.fit_transform(korpus)

# Ambil nama-nama kata (fitur/term)
feature_names = cv.get_feature_names_out()

# Ubah matriks hasil menjadi DataFrame untuk tampilan yang lebih rapi
df_count = pd.DataFrame(word_count_matrix.toarray(),
                        index=[f"Komentar {i+1}" for i in range(len(korpus))],
                        columns=feature_names)

print("\n## Hasil Matriks Frekuensi Kata (Term Frequency Matrix)")
print(df_count)
print("-" * 50)


## Hasil Matriks Frekuensi Kata (Term Frequency Matrix)
            admin  bagus  bahasa  banget  dimengerti  gampang  info  kasih  \
Komentar 1      0      0       0       1           0        0     1      1   
Komentar 2      0      0       0       0           0        0     0      0   
Komentar 3      0      1       1       0           1        0     0      0   
Komentar 4      1      0       0       0           0        0     0      0   
Komentar 5      0      0       0       1           0        1     0      0   

            keren  konten  pendapat  poin  saran  setuju  tantangan  terima  \
Komentar 1      1       0         0     0      0       0          0       1   
Komentar 2      0       0         1     1      0       1          0       0   
Komentar 3      0       1         0     0      1       0          0       0   
Komentar 4      0       0         0     0      0       0          0       0   
Komentar 5      0       0         0     0      0       0          1       0   


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [11]:
# =================================================================
# LANGKAH 3: TERM FREQUENCY-INVERSE DOCUMENT FREQUENCY (TF-IDF)
# =================================================================

# Inisialisasi TfidfVectorizer dengan stopwords Bahasa Indonesia
tfidf_vectorizer = TfidfVectorizer(stop_words=list_stopwords)

# Terapkan TfidfVectorizer ke korpus
tfidf_matrix = tfidf_vectorizer.fit_transform(korpus)

# Ambil nama-nama kata (fitur/term)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Ubah matriks hasil menjadi DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(),
                        index=[f"Komentar {i+1}" for i in range(len(korpus))],
                        columns=tfidf_feature_names)

print("\n## Hasil Matriks TF-IDF")
print(df_tfidf)
print("-" * 50)


## Hasil Matriks TF-IDF
              admin     bagus    bahasa    banget  dimengerti   gampang  \
Komentar 1  0.00000  0.000000  0.000000  0.339393    0.000000  0.000000   
Komentar 2  0.00000  0.000000  0.000000  0.000000    0.000000  0.000000   
Komentar 3  0.00000  0.447214  0.447214  0.000000    0.447214  0.000000   
Komentar 4  0.57735  0.000000  0.000000  0.000000    0.000000  0.000000   
Komentar 5  0.00000  0.000000  0.000000  0.495524    0.000000  0.614189   

                info     kasih     keren    konten  pendapat     poin  \
Komentar 1  0.420669  0.420669  0.420669  0.000000   0.00000  0.00000   
Komentar 2  0.000000  0.000000  0.000000  0.000000   0.57735  0.57735   
Komentar 3  0.000000  0.000000  0.000000  0.447214   0.00000  0.00000   
Komentar 4  0.000000  0.000000  0.000000  0.000000   0.00000  0.00000   
Komentar 5  0.000000  0.000000  0.000000  0.000000   0.00000  0.00000   

               saran   setuju  tantangan    terima   tunggu   upload     video  
Kome

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
